In [ ]:
!pip install -q -U accelerate peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install fsspec==2025.3.2
# !pip install fsspec==2024.12.0
# !pip install gcsfs==2024.12.0


In [ ]:
# !pip install transformers

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    AutoModelForSequenceClassification
)


In [ ]:
import pandas as pd

# data1 = pd.read_csv('/content/hindi_english_sentiment_reviews.csv')
# data2 = pd.read_csv("/content/pure_english_sentiment_conversations.csv")
data3 = pd.read_csv("/content/call_conversation_sentences_1500.csv")

In [ ]:
data = data3

In [ ]:
data.head()

,text,label
0,Appreciate the fast response.,5
1,The staff was very polite and helpful.,4
2,कर्मचारी बहुत विनम्र और सहायक थे।,4
3,"Hello, I am from customer support, how may I a...",5
4,The service was not helpful at all.,1


In [ ]:
data.to_csv('sample.csv')

In [ ]:
# df = pd.read_csv("sample.csv")
# df = df.iloc[:, 1:]

In [ ]:
data

,text,label
0,Appreciate the fast response.,5
1,The staff was very polite and helpful.,4
2,कर्मचारी बहुत विनम्र और सहायक थे।,4
3,"Hello, I am from customer support, how may I a...",5
4,The service was not helpful at all.,1
...,...,...
1495,मुझे अपने फोन के पावर बटन में समस्या है।,4
1496,"नमस्ते, मैं ग्राहक सेवा से बोल रहा हूँ, मैं आप...",5
1497,"नमस्ते, मैं ग्राहक सेवा से बोल रहा हूँ, मैं आप...",5
1498,Appreciate the fast response.,5


In [ ]:
import json

dataset = data.to_dict(orient="records")

with open("data.jsonl", "w", encoding="utf-8") as f:
    for row in dataset:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")



In [ ]:
# dataset

In [ ]:
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

In [ ]:
from datasets import Dataset

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(dataset)

# Optional: Convert 1–5 labels to 0–4 if using nlptown or similar models
dataset = dataset.map(lambda x: {"label": x["label"] - 1})

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1500
})

In [ ]:
# import os
# from huggingface_hub import login

# hf_token = os.getenv("hf_***********************************")  # Retrieve from secrets
# login(hf_token)


In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
# model_name = "tabularisai/multilingual-sentiment-analysis"
dataset_name = dataset
lora_r = 128

lora_alpha = 32

lora_dropout = 0.05
use_4bit = True


bnb_4bit_compute_dtype = "float16"

bnb_4bit_quant_type = "nf4"

use_nested_quant = False

output_dir = "./results"

num_train_epochs = 3

fp16 = False
bf16 = False

per_device_train_batch_size = 2

per_device_eval_batch_size = 2

gradient_accumulation_steps = 2

gradient_checkpointing = True

max_grad_norm = 1.0

learning_rate = 1e-5

weight_decay = 0.001

optim = "paged_adamw_32bit"

lr_scheduler_type = "cosine"

max_steps = 600

warmup_ratio = 0.1

group_by_length = True

save_steps = 500

logging_steps = 50

max_seq_length = 4096

packing = True

device_map = {"": 0}

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1500
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Configure BitsAndBytes
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Split dataset
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Define preprocessing function
def preprocess_function(examples):
    # Tokenize texts
    text_column = "text"
    label_column = "label"

    tokenized = tokenizer(
        examples[text_column],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Add labels
    tokenized["labels"] = examples[label_column]
    return tokenized

# Apply preprocessing
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

# Convert to PyTorch tensors
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    num_labels=5
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Define LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    target_modules=["query", "key", "value"]  # Adjust based on your model
)

# Get PEFT model
model = get_peft_model(model, peft_config)

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    optim=optim,
    fp16=fp16,
    bf16=bf16,
    logging_steps=logging_steps,
    save_steps=save_steps,
    gradient_accumulation_steps=gradient_accumulation_steps,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# Start training
trainer.train()

Map:   0%|          | 0/1350 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

{'loss': 1.1945, 'grad_norm': 2.5279035568237305, 'learning_rate': 8.166666666666668e-06, 'epoch': 0.14814814814814814}
{'loss': 1.1747, 'grad_norm': 1.944199800491333, 'learning_rate': 9.871850323926178e-06, 'epoch': 0.2962962962962963}
{'loss': 1.1521, 'grad_norm': 5.356739044189453, 'learning_rate': 9.344598067954151e-06, 'epoch': 0.4444444444444444}
{'loss': 1.0923, 'grad_norm': 4.303302764892578, 'learning_rate': 8.452308444726249e-06, 'epoch': 0.5925925925925926}
{'loss': 1.0678, 'grad_norm': 3.7668306827545166, 'learning_rate': 7.269952498697734e-06, 'epoch': 0.7407407407407407}
{'loss': 1.0046, 'grad_norm': 3.76147198677063, 'learning_rate': 5.896872929687287e-06, 'epoch': 0.8888888888888888}
{'loss': 0.9883, 'grad_norm': 5.493687152862549, 'learning_rate': 4.448437222342441e-06, 'epoch': 1.0355555555555556}
{'loss': 0.9565, 'grad_norm': 3.4718635082244873, 'learning_rate': 3.0463443575536324e-06, 'epoch': 1.1837037037037037}
{'loss': 0.923, 'grad_norm': 3.27445387840271, 'lear

TrainOutput(global_step=600, training_loss=1.027945556640625, metrics={'train_runtime': 90.2421, 'train_samples_per_second': 26.595, 'train_steps_per_second': 6.649, 'train_loss': 1.027945556640625, 'epoch': 1.7762962962962963})

In [ ]:
for i in range(3):
    print(dataset[i])

{'text': 'Appreciate the fast response.', 'label': 4}
{'text': 'The staff was very polite and helpful.', 'label': 3}
{'text': 'कर्मचारी बहुत विनम्र और सहायक थे।', 'label': 3}


In [ ]:
# for item in dataset:
#     item["label"] -= 1

# # Tokenize dataset
# def tokenize(example):
#     return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# tokenized_dataset = dataset.map(tokenize)
# tokenized_dataset

In [ ]:
labels = []
for i in range(1500):
  labels.append(dataset[i]['label'])

print(min(labels))

0


In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1500
})

In [ ]:

%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
from transformers import pipeline, logging
logging.set_verbosity(logging.CRITICAL)

# Assuming `model` and `tokenizer` are already loaded
prompt = "Tis was excelent"
formatted_prompt = f"[INST] {prompt} [/INST]"

pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
result = pipe(formatted_prompt)

print(result[0])

In [ ]:
from transformers import pipeline
import pandas as pd

# Load the model
pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

# Label-to-rating mapping
label_to_rating = {
    "very negative": 0,
    "negative": 1,
    "slightly negative": 2,
    "neutral": 3,
    "positive": 4,
    "very positive": 5
}
# Convert to DataFrame and save as CSV
df = pd.DataFrame(results)
df.to_csv("hindi_sentiment_predictions.csv", index=False)

# Download CSV in Colab
from google.colab import files
files.download("hindi_sentiment_predictions.csv")


config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

176

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Save the merged model (full model without adapter)
model.save_pretrained("merged_finetuned_model")
tokenizer.save_pretrained("merged_finetuned_model")


print("Full model saved successfully!")


Full model saved successfully!


In [ ]:
!zip -r merged_finetuned_model.zip merged_finetuned_model

  adding: merged_finetuned_model/ (stored 0%)
  adding: merged_finetuned_model/model-00002-of-00003.safetensors


zip error: Interrupted (aborting)



In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# import torch

# # Load the full model (after merging LoRA)
# model_path = "./final_model"
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_path)

# # Initialize text generation pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1000)

# # Example prompt
# prompt = "acne treatment"
# result = pipe(f"[INST] {prompt} [/INST]")

# # Print the generated text
# print(result[0]['generated_text'])


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
# model.push_to_hub("santanukumar07/sentiment")
# tokenizer.push_to_hub("santanukumar07/sentiment")
